In [1]:
import pandas as pd
import re
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn import datasets, svm, metrics

In [ ]:
# removing depth and table ONLY and also taking out outliers but NOT scaling or creating volumen column

In [2]:
diamonds=pd.read_csv("../input/data.csv")
diamonds.shape


(40455, 10)

In [3]:
#"Dropping depth and table"
def clean_transformed(df):
    ''' applicable for test and data sets: transforms categorical variables cut, color and clarity, creates new variable volumen, drops x,y,z and 
        depth and table.
    '''
    df['cut']=df['cut'].map({'Fair':0,'Good':1,'Very Good': 2,'Premium':3,'Ideal':4})
    df['color']=df['color'].map({'J':0,'I':1,'H':2,'G':3,'F':4,'E':5,'D':6})
    df['clarity']=df['clarity'].map({'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7})
    df =df.drop(['depth','table'], axis=1)
    return df

In [4]:
diamonds1= clean_transformed(diamonds)
diamonds1

,carat,cut,color,clarity,x,y,z,price
0,2.26,4,3,1,8.44,8.36,5.20,12831
1,2.43,2,2,1,8.56,8.50,5.39,16170
2,0.80,3,4,1,6.03,6.01,3.67,2797
3,0.40,4,4,0,4.68,4.64,2.95,630
4,0.31,4,3,3,4.39,4.37,2.70,698
...,...,...,...,...,...,...,...,...
40450,1.11,3,2,2,6.63,6.56,4.14,5315
40451,0.73,4,4,3,5.77,5.74,3.60,2762
40452,1.26,2,1,4,7.09,7.02,4.18,6855
40453,0.72,4,3,1,5.76,5.83,3.56,2297


In [5]:
def remove_outliers(df):
    '''only applicable to the data set NOT the test set'''
    z = np.abs(stats.zscore(df))
    df =df[(z<3).all(axis=1)]
    return df


In [6]:
diamonds2= remove_outliers(diamonds1)
diamonds2

,carat,cut,color,clarity,x,y,z,price
2,0.80,3,4,1,6.03,6.01,3.67,2797
3,0.40,4,4,0,4.68,4.64,2.95,630
4,0.31,4,3,3,4.39,4.37,2.70,698
5,0.53,2,4,1,5.19,5.23,3.15,1132
6,0.70,3,6,1,5.69,5.64,3.45,1987
...,...,...,...,...,...,...,...,...
40450,1.11,3,2,2,6.63,6.56,4.14,5315
40451,0.73,4,4,3,5.77,5.74,3.60,2762
40452,1.26,2,1,4,7.09,7.02,4.18,6855
40453,0.72,4,3,1,5.76,5.83,3.56,2297


In [7]:
diamonds_test=pd.read_csv("../input/test.csv",index_col='id')
diamonds_test

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,0.56,Ideal,I,VS2,62.1,54.0,5.30,5.33,3.30
1,1.05,Ideal,G,VS2,61.9,56.0,6.56,6.52,4.05
2,0.50,Premium,E,VS2,61.5,56.0,5.11,5.07,3.13
3,0.80,Ideal,F,VS1,62.8,56.0,5.91,5.96,3.73
4,0.54,Ideal,G,VS1,61.4,57.0,5.23,5.20,3.20
...,...,...,...,...,...,...,...,...,...
13480,0.70,Premium,H,SI1,59.0,60.0,5.86,5.81,3.44
13481,0.55,Ideal,G,VVS2,62.3,53.0,5.27,5.29,3.29
13482,1.54,Ideal,G,SI1,61.5,55.0,7.40,7.45,4.57


In [8]:
diamonds_test1= clean_transformed(diamonds_test)
diamonds_test1

,carat,cut,color,clarity,x,y,z
id,,,,,,,
0,0.56,4,1,3,5.30,5.33,3.30
1,1.05,4,3,3,6.56,6.52,4.05
2,0.50,3,5,3,5.11,5.07,3.13
3,0.80,4,4,4,5.91,5.96,3.73
4,0.54,4,3,4,5.23,5.20,3.20
...,...,...,...,...,...,...,...
13480,0.70,3,2,2,5.86,5.81,3.44
13481,0.55,4,3,5,5.27,5.29,3.29
13482,1.54,4,3,2,7.40,7.45,4.57


In [9]:
#only for data set original

def modelling (df):
    ''' Only applicable to original data set '''
    X=df.drop(['price'],axis=1)
    y=df['price']
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
    
    models = {
        "decision_tree": DecisionTreeRegressor(random_state = 0), #ok
        "forest": RandomForestRegressor(n_estimators=100), #ok
        "neighbors": KNeighborsRegressor(n_neighbors=2),#ok
        "gradient boosting": GradientBoostingRegressor() #ok
    
        }

    for modelName, model in models.items():
        print(f"Training model: {modelName}")
        model.fit(X_train, y_train)
    
    d = {modelName:model.predict(X_test) for modelName, model in models.items()}

    df = pd.DataFrame(d)
    df["gt"] = y_test.reset_index(drop=True)
    return df

In [10]:
#model predicted 

diamonds3 = modelling (diamonds2)
diamonds3

Training model: decision_tree
Training model: forest
Training model: neighbors
Training model: gradient boosting


,decision_tree,forest,neighbors,gradient boosting,gt
0,410.0,491.07,381.5,241.602035,526
1,765.0,720.18,765.0,759.321934,663
2,932.0,912.86,815.0,825.118580,962
3,463.0,485.36,424.0,388.334482,434
4,5290.0,5533.47,5761.5,5403.103998,5290
...,...,...,...,...,...
7875,737.0,735.12,746.0,880.651199,673
7876,1817.0,1457.62,1495.5,1358.613369,1401
7877,1031.0,1052.51,959.5,944.239067,1000
7878,975.0,981.46,975.0,1000.965775,975


In [11]:
results2=metrics.mean_squared_error(diamonds3['gt'], diamonds3['decision_tree'], squared=True)
results2
print(f' mean square error of decision tree is {results2}')

 mean square error of decision tree is 368169.68381979695


In [12]:
results4=metrics.mean_squared_error(diamonds3['gt'], diamonds3['forest'], squared=True)
results4
print(f' mean square error of forest is {results4}')

 mean square error of forest is 203595.68715404443


In [13]:
results4**(1/2)

451.21578779342866

In [14]:
results6=metrics.mean_squared_error(diamonds3['gt'], diamonds3['neighbors'], squared=True)
results6
print(f' mean square error of neighbors is {results6}')

 mean square error of neighbors is 319630.4608185279


In [15]:
results8=metrics.mean_squared_error(diamonds3['gt'], diamonds3['gradient boosting'], squared=True)
results8
print(f' mean square error of gradient is {results8}')

 mean square error of gradient is 254886.41421851574


In [17]:

# to calculate prediction for test ONLY 
models = {
        "decision_tree": DecisionTreeRegressor(random_state = 0), #ok
        "forest": RandomForestRegressor(n_estimators=100), #ok
        "neighbors": KNeighborsRegressor(n_neighbors=2),#ok
        "gradient boosting": GradientBoostingRegressor() #ok
        }

def modeling(df, model_type):
    ''' only applies to original data set'''
    X=df.drop(['price'],axis=1)
    y=df['price']
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
    model = model_type
    model.fit(X_train, y_train)
    return model

def predict(df, model):
    ''' works for test only'''
    d = model.predict(df)
    df = pd.DataFrame(d)
    return df

# to obtain predicions for test set in predict use test and in modeling use data set
for model_name, model in models.items(): # 1) iterar por cada tipo de modelo
    trained_model = modeling(diamonds2, model) # 2) data, model entrenar el modelo con data original, añadir métricas
    prediction = predict(diamonds_test1, trained_model) # 3) prediccion en test con el modelo entrenado
    #print (model_name)
    prediction= prediction.rename(columns={0: "price"})
    prediction.index.name= 'id'
    prediction.to_csv(f'../output/{model_name} test_no_scalev1.csv')
    print(prediction)

    

         price
id            
0       1417.0
1       6471.0
2       1746.0
3       4412.0
4       1567.0
...        ...
13480   1858.0
13481   1981.0
13482  11230.0
13483    506.0
13484   1167.0

[13485 rows x 1 columns]
          price
id             
0       1379.22
1       6165.47
2       1708.95
3       3992.41
4       1741.42
...         ...
13480   2293.99
13481   2128.01
13482  11070.53
13483    495.22
13484   1226.78

[13485 rows x 1 columns]
         price
id            
0       1368.0
1       6634.0
2       1746.0
3       4272.5
4       1770.0
...        ...
13480   2188.5
13481   2022.5
13482  12480.0
13483    506.0
13484   1243.0

[13485 rows x 1 columns]
              price
id                 
0       1432.652106
1       6324.375984
2       1612.017728
3       3807.091917
4       1731.620689
...             ...
13480   2239.722873
13481   1962.735235
13482  11422.106418
13483    378.890501
13484   1265.596042

[13485 rows x 1 columns]
